<a href="https://colab.research.google.com/github/bnsreenu/python_for_microscopists/blob/master/293_denoising_RGB_images_using_deep%20learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/71wqPyapFGU

This example is for training a Noise2Void denoising model using RGB images. Images need to be in lossless format (e.g., png or tiff). JPG format is not allowed if you're using the DataGenerator object but you can use skimage or other ibraries to load them. The same approach can be used for grey-scale images, for example SEM or CT images.

In [ ]:
#Install the tensorflow library suggested by N2V.
!pip install tensorflow==2.12.0
!pip install n2v

# Mount Google Drive to have access to your data which you should store there

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import n2v
print(tf.__version__)
print(n2v.__version__)

2.12.0
0.3.2


In [ ]:
# We import all our dependencies.
from n2v.models import N2VConfig, N2V
import numpy as np
from csbdeep.utils import plot_history
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator

from matplotlib import pyplot as plt
import urllib
import os
import zipfile

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# We import all our dependencies.
from n2v.models import N2V
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.image import imread, imsave
from csbdeep.io import save_tiff_imagej_compatible

In [ ]:
# We create our DataGenerator-object.
# It will help us load data and extract patches for training and validation.
datagen = N2V_DataGenerator()

If you have many images stored in a directory, use load_imgs_from_directory method to load them into a list. Images can be of different sizes as we are capturing them into a list, not an array.
<p>
You can also use your favorite way of reading images (e.g., using skimage) sequentially and capturing them into a list that can be used as input to the network.
<p>
Note that the images need to be in float32 format. load_imgs_from_directory method converts images to float32 but if you use your own way of loading images, you need to make sure they are converted to float32.

In [ ]:
def NormalizeImg(img):
    """Normalize img input by subtracting the mean
    and dividing by std"""
    img = img.astype("float32")
    img -= np.mean(img)
    img /= np.std(img)

    return img

In [ ]:
def n2vDenoising(img, visualize=False, pass_it=False):
    img_normalized = NormalizeImg(img)
    if pass_it:
        return img_normalized

    # We import all our dependencies.
    model_name = "n2v_transwell"
    model_dir = "/content/drive/MyDrive/Lucas-Drive-Foldername/Noise2Void/models"
    model = N2V(config=None, name=model_name, basedir=model_dir)

    # change shape to (y,x, c=1)
    img_normalized = np.expand_dims(img_normalized, axis=-1)

    pred = model.predict(img_normalized, axes="YXC")

    if visualize:
        fig, axs = plt.subplots(1, 2, figsize=(10, 10))

        axs[0].imshow(img)
        axs[0].set_title("input")

        axs[1].imshow(pred[:, :, 0])
        axs[1].set_title("output")

        plt.show()

    return pred[:, :, 0]  # return same shape as input


In [ ]:
def GetTranswellData(base_dir, competition, files_are_in):
    different_assays_folders = [
        file for file in os.listdir(base_dir + competition + files_are_in) if file.endswith(".tif")
    ]

    # get all files that start with WT_
    WT_files = [file for file in different_assays_folders if file.startswith("WT_")]

    Mutant_files = [file for file in different_assays_folders if not file.startswith("WT_")]

    # sort the files
    WT_files.sort()
    Mutant_files.sort()

    return WT_files, Mutant_files

<h1>Denoising images using the trained model</h1>

In [ ]:
# collect all data

base_dir = "/content/drive/MyDrive/Lucas-Drive-Foldername/dataset_competition_assays/"

competition = "competition_2_WTmScarlet_dwspFmNeonGreen/"
files_are_in = "TW_growth/"



input_dir = base_dir + competition + files_are_in
output_dir = input_dir + "denoised/"

# create output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# get all filenames
WT_files, Mutant_files = GetTranswellData(
    base_dir, competition, files_are_in
)

## Run the denoising script

In [ ]:
from skimage.io import imread,imsave
# TODO: this is just mock code, because "filenames" does not exist yet and data should also be stored in the correct subfolders ideally
for WT_fn, Mutant_fn in zip(WT_files, Mutant_files):


    # process WT
    img = imread(input_dir + WT_fn)
    denoised_data = n2vDenoising(img, visualize=False, pass_it=False)
    imsave(output_dir + WT_fn,denoised_data)

    # process Mutant
    img = imread(input_dir + Mutant_fn)
    denoised_data = n2vDenoising(img, visualize=False, pass_it=False)
    imsave(output_dir + Mutant_fn,denoised_data)



print("The results are now in " + output_dir + " in your google drive")

float32
(250, 256, 256)
